In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
spark = (
    SparkSession
    .builder
    .master("local[*]")
    .appName("spark_sql")
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
24/07/29 11:35:31 WARN Utils: Your hostname, Home-PC resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/07/29 11:35:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/29 11:35:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = (
    spark
    .read
    .option('delimiter', ';')
    .option('header', 'true')
    .option('inferSchema', 'true')
    .option('enconding', 'ISO-8859-1')
    .csv('./dados/precos-gasolina-etanol-04.csv')
)

In [4]:
df.printSchema()

root
 |-- Regiao - Sigla: string (nullable = true)
 |-- Estado - Sigla: string (nullable = true)
 |-- Municipio: string (nullable = true)
 |-- Revenda: string (nullable = true)
 |-- CNPJ da Revenda: string (nullable = true)
 |-- Nome da Rua: string (nullable = true)
 |-- Numero Rua: string (nullable = true)
 |-- Complemento: string (nullable = true)
 |-- Bairro: string (nullable = true)
 |-- Cep: string (nullable = true)
 |-- Produto: string (nullable = true)
 |-- Data da Coleta: string (nullable = true)
 |-- Valor de Venda: string (nullable = true)
 |-- Valor de Compra: string (nullable = true)
 |-- Unidade de Medida: string (nullable = true)
 |-- Bandeira: string (nullable = true)



In [5]:
df.createOrReplaceTempView('combustiveis')

In [6]:
spark.sql("""
    select `Estado - Sigla`, `Produto`, `Valor de Venda`, `Unidade de Medida` from combustiveis
""").show()

+--------------+------------------+--------------+-----------------+
|Estado - Sigla|           Produto|Valor de Venda|Unidade de Medida|
+--------------+------------------+--------------+-----------------+
|            SP|          GASOLINA|          4,88|       R$ / litro|
|            SP|            ETANOL|          3,09|       R$ / litro|
|            RS|          GASOLINA|          5,49|       R$ / litro|
|            RS|GASOLINA ADITIVADA|          5,49|       R$ / litro|
|            RS|            ETANOL|          4,39|       R$ / litro|
|            AC|          GASOLINA|          7,33|       R$ / litro|
|            AC|GASOLINA ADITIVADA|          7,49|       R$ / litro|
|            AC|          GASOLINA|          7,33|       R$ / litro|
|            AC|GASOLINA ADITIVADA|          7,43|       R$ / litro|
|            AC|            ETANOL|          5,79|       R$ / litro|
|            AC|          GASOLINA|          6,79|       R$ / litro|
|            AC|GASOLINA ADITIVADA

In [7]:
view_precos = spark.sql("""
    select 
        `Estado - Sigla`
        ,`Produto`
        ,regexp_replace(`Valor de Venda`, ',', '.') as `Valor de Venda`
        ,`Unidade de Medida` 
        from combustiveis
""")

In [8]:
view_precos.show(10)

+--------------+------------------+--------------+-----------------+
|Estado - Sigla|           Produto|Valor de Venda|Unidade de Medida|
+--------------+------------------+--------------+-----------------+
|            SP|          GASOLINA|          4.88|       R$ / litro|
|            SP|            ETANOL|          3.09|       R$ / litro|
|            RS|          GASOLINA|          5.49|       R$ / litro|
|            RS|GASOLINA ADITIVADA|          5.49|       R$ / litro|
|            RS|            ETANOL|          4.39|       R$ / litro|
|            AC|          GASOLINA|          7.33|       R$ / litro|
|            AC|GASOLINA ADITIVADA|          7.49|       R$ / litro|
|            AC|          GASOLINA|          7.33|       R$ / litro|
|            AC|GASOLINA ADITIVADA|          7.43|       R$ / litro|
|            AC|            ETANOL|          5.79|       R$ / litro|
+--------------+------------------+--------------+-----------------+
only showing top 10 rows



In [9]:
view_precos.createOrReplaceTempView('view_precos')

In [10]:
df_diferenca_precos = spark.sql("""
    select 
        `Estado - Sigla`
        ,`Produto`
        ,`Unidade de Medida`
        ,MIN(`Valor de Venda`) as menor_valor
        ,MAX(`Valor de Venda`) as maior_valor
        ,MAX(`Valor de Venda`) - MIN(`Valor de Venda`) as diferenca
    from view_precos
    group by all
    order by diferenca DESC
""")

In [11]:
df_diferenca_precos.show()

+--------------+------------------+-----------------+-----------+-----------+------------------+
|Estado - Sigla|           Produto|Unidade de Medida|menor_valor|maior_valor|         diferenca|
+--------------+------------------+-----------------+-----------+-----------+------------------+
|            SP|GASOLINA ADITIVADA|       R$ / litro|       4.69|       8.89|               4.2|
|            SP|          GASOLINA|       R$ / litro|       4.69|       7.97|3.2799999999999994|
|            RJ|GASOLINA ADITIVADA|       R$ / litro|       5.29|       7.79|               2.5|
|            SP|            ETANOL|       R$ / litro|       2.85|       5.29|              2.44|
|            RJ|          GASOLINA|       R$ / litro|       5.19|       7.59|2.3999999999999995|
|            PE|            ETANOL|       R$ / litro|       3.39|       5.69|2.3000000000000003|
|            PR|            ETANOL|       R$ / litro|       3.17|       5.39|2.2199999999999998|
|            RS|            ET